* simple Named Entity Reongition model with VAR and TYPE tags using spaCy
* training data: tex files from the Stacks Project annotated using a "Let ... be a ..." rule
* inspired by https://github.com/explosion/spaCy/blob/master/examples/training/train_ner.py
* use regular expression + noun chunk   rule

In [181]:
from __future__ import unicode_literals, print_function
import json
import pathlib
import random

import spacy
from spacy.pipeline import EntityRecognizer
from spacy.gold import GoldParse
from spacy.tagger import Tagger
from spacy.matcher import Matcher
from spacy.attrs import IS_PUNCT, LOWER, TAG, ORTH

import os
import re
 
try:
    unicode
except:
    unicode = str

In [182]:
nlp = spacy.load('en')

In [183]:
#initialize the triggering rule 'let $' for the rule-based annotation
matcher = Matcher(nlp.vocab)
matcher.add_pattern("let", [{LOWER: "let"}, {ORTH: "$"}])

In [184]:
def tex2doc(tex_file):  #read the whole tex file in the spaCy doc object
    with open(tex_file, 'r') as tex:
        data=tex.read()
    doc = nlp(data)
    return doc

In [185]:
def rule_based_annotation(doc):
    annotation = []
    for m in matcher(doc):
        let_position = m[2]
        dollar_position = m[3]
        annotation += add_letDollarBe_entity(doc, let_position, dollar_position)
    return (doc.text, annotation)


def add_letDollarBe_entity(doc, let_position, dollar_position):
    new_annotation=[]
    sentence = doc[let_position : dollar_position].sent   #stay in the current sentence, so to respect sentence boundaries
    print('===== sent', sentence, '========\n')
    
    part = doc[let_position : sentence.end]  #forget the already-processed first part of the sentence
    
    for regex_match in re.finditer('^let (\$+[^\$]+\$+) be (an?|the) \S+', part.text, re.IGNORECASE): #there is at most one match in this for loop because of ^
        #compute left and right char offset for candidate VAR
        left_offset = part.start_char + regex_match.span(1)[0]
        right_offset = part.start_char + regex_match.span(1)[1]
        
        print('regex_match', regex_match.group(1))
        for nn in part.noun_chunks:
            print('nn candidate', nn)
            if nn.start_char >= right_offset + 3: #we only consider noun chunks after the '$ be '
                
                
                
                new_annotation += ((left_offset, right_offset, 'VAR'),)
                
                #left and right char offset for TYPE are already among the nn attributes
                new_annotation += ((nn.start_char, nn.end_char, 'TYPE'),)
                print('nn', nn.text)
                break  #we only consider the 1st noun chunk after the 'be' 
                
    
    print('new_annotation', new_annotation)
                    
    return new_annotation

In [244]:
annotated_data=[]

directory = os.fsencode('tex_files/')
list_of_texs = [os.fsdecode(file) for file in os.listdir(directory)[0:10]]  #remove [0:10] to get the full data
for filename in list_of_texs:
    print("file: ", filename)
    doc = tex2doc(os.path.join(os.fsdecode(directory), filename))
    annotated_data.append(rule_based_annotation(doc))    

file:  pione.tex
===== sent Before we state the result we introduce the category of $G$-sets for a
topological group $G$.

\begin{definition}
\label{definition-G-set-continuous}
Let $G$ be a topological group.

regex_match $G$
nn candidate a topological group
nn a topological group
new_annotation [(794, 797, 'VAR'), (801, 820, 'TYPE')]
===== sent \medskip\noindent
Recall that if $L/K$ is an infinite Galois extension then the
Galois group $G = \text{Gal}(L/K)$ comes endowed with a canonical
topology, see Fields, Section \ref{fields-section-infinite-galois}.

\begin{lemma}
\label{lemma-sheaves-point}
Let $K$ be a field. ========

regex_match $K$
nn candidate a field
nn a field
new_annotation [(1863, 1866, 'VAR'), (1870, 1877, 'TYPE')]
===== sent Let $K^{sep}$ a separable closure of $K$.
Consider the profinite group $G = \text{Gal}(K^{sep}/K)$.

new_annotation []
===== sent In this section we discuss some of the material the reader can
find in \cite[Expos\'e V, Sections 4, 5, and 6]{SGA1}

===== sent Let $B$ be the integral closure of $A$ in $L$.
Let $\mathfrak m$ be a maximal ideal of $B$.
Let $G = \text{Gal}(L/K)$,
$D = \{\sigma \in G \mid \sigma(\mathfrak m) = \mathfrak m\}$, and
$I = \{\sigma \in D \mid \sigma \bmod \mathfrak m =
\text{id}_{\kappa(\mathfrak ========

regex_match $\mathfrak m$
nn candidate a maximal ideal
nn a maximal ideal
new_annotation [(122557, 122570, 'VAR'), (122574, 122589, 'TYPE')]
===== sent Let $B$ be the integral closure of $A$ in $L$.
Let $\mathfrak m$ be a maximal ideal of $B$.
Let $G = \text{Gal}(L/K)$,
$D = \{\sigma \in G \mid \sigma(\mathfrak m) = \mathfrak m\}$, and
$I = \{\sigma \in D \mid \sigma \bmod \mathfrak m =
\text{id}_{\kappa(\mathfrak ========

new_annotation []
===== sent To construct the second short exact sequence let $\Lambda$ be the set
of finite Galois subextensions, i.e., $\lambda \in \Lambda$ corresponds
to $L/L_\lambda/K$. Set $G_\lambda = \text{Gal}(L_\lambda/K)$.
Recall that $G_\lambda$ is an inverse system of fin

===== sent \medskip\noindent
To get rid of the ramification we are going to choose a further finite
separable extension $K^{sep}/L'/L/K$ such that the ramification
index $e$ of the induced extensions $B'/B$ is divisible by $e_i$.
Consider the normalized base change $Z'$ of $Z$ with respect to
$\Spec(B') \to \Spec(B)$, see discussion in
More on Morphisms, Section \ref{more-morphisms-section-reduced-fibre-theorem}.
Let $\xi_{i, j}$ be the points of $Z'$ mapping to $\xi_{B'}$
and to $\xi_i$ in $Z$. ========

regex_match $\xi_{i, j}$
nn candidate the points
nn the points
new_annotation [(212135, 212147, 'VAR'), (212151, 212161, 'TYPE')]
===== sent \end{proof}

\noindent
Let $G$ be a profinite group. ========

regex_match $G$
nn candidate a profinite group
nn a profinite group
new_annotation [(213259, 213262, 'VAR'), (213266, 213283, 'TYPE')]
===== sent Let $p$ be a prime number.

regex_match $p$
nn candidate a prime number
nn a prime number
new_annotation [(213289, 213292, 'VAR'), (213296,

file:  spaces-simplicial.tex
===== sent \medskip\noindent
Let $X$ be a simplicial space. ========

regex_match $X$
nn candidate a simplicial space
nn a simplicial space
new_annotation [(1804, 1807, 'VAR'), (1811, 1829, 'TYPE')]
===== sent \begin{lemma}
\label{lemma-simplicial-site}
Let $X$ be a simplicial space. ========

regex_match $X$
nn candidate a simplicial space
nn a simplicial space
new_annotation [(2935, 2938, 'VAR'), (2942, 2960, 'TYPE')]
===== sent Let $X$ be a simplicial space. ========

regex_match $X$
nn candidate a simplicial space
nn a simplicial space
new_annotation [(3068, 3071, 'VAR'), (3075, 3093, 'TYPE')]
===== sent Let $\mathcal{F}$ be a sheaf on $X_{Zar}$.
It is clear from the definition of coverings, that the restriction
of $\mathcal{F}$ to the opens of $X_n$ defines a sheaf $\mathcal{F}_n$
on the topological space $X_n$. For every $\varphi : [m] \to [n]$ the
restriction maps of $\mathcal{F}$ for pairs $U \subset X_n$, $V \subset X_m$
with $X(\varphi)(U) \subset

nn candidate a hypercovering
nn a hypercovering
new_annotation [(176026, 176029, 'VAR'), (176033, 176048, 'TYPE')]
===== sent \begin{lemma}
\label{lemma-hypercovering-X-descent-sheaves}
Let $\mathcal{C}$ be a site with fibre products and $X \in \Ob(\mathcal{C})$.
Let $K$ be a hypercovering of $X$. Then
\begin{enumerate}
\item $a^{-1} : \Sh(\mathcal{C}/X) \to \Sh((\mathcal{C}/K)_{total})$
is fully faithful with essential image the cartesian sheaves of sets,
\item $a^{-1} : \textit{Ab}(\mathcal{C}/X) \to
\textit{Ab}((\mathcal{C}/K)_{total})$
is fully faithful with essential image the cartesian sheaves
of abelian groups.

regex_match $\mathcal{C}$
nn candidate a site
nn a site
new_annotation [(177356, 177369, 'VAR'), (177373, 177379, 'TYPE')]
===== sent \begin{lemma}
\label{lemma-hypercovering-X-descent-sheaves}
Let $\mathcal{C}$ be a site with fibre products and $X \in \Ob(\mathcal{C})$.
Let $K$ be a hypercovering of $X$. Then
\begin{enumerate}
\item $a^{-1} : \Sh(\mathcal{C}/X) \to \Sh(

...

nn candidate $
nn candidate X \to
nn candidate a morphism
nn a morphism
new_annotation [(168836, 168849, 'VAR'), (168853, 168863, 'TYPE')]
===== sent Let $f : X \to Y$ be a morphism of schemes over $S$.
Let $\mathcal{F}$ be a quasi-coherent $\mathcal{O}_X$-module.

regex_match $\mathcal{F}$
nn candidate a quasi-coherent $\mathcal{O}_X$-module
nn a quasi-coherent $\mathcal{O}_X$-module
new_annotation [(168889, 168902, 'VAR'), (168906, 168945, 'TYPE')]
===== sent Y'_{s'} \ar[r] & Y_s
}
$$
the horizontal morphisms are flat as they are base changes by the flat
morphism $\Spec(\kappa(s')) \to \Spec(\kappa(s))$.
\end{proof}

\begin{lemma}
\label{lemma-base-change-flatness-fibres}
Let $S$ be a scheme. ========

regex_match $S$
nn candidate a scheme
nn a scheme
new_annotation [(170848, 170851, 'VAR'), (170855, 170863, 'TYPE')]
===== sent Let $f : X \to Y$ be a morphism of schemes over $S$.
Assume
\begin{enumerate}
\item $X$ is locally of finite presentation over $S$,
\item $X$ is flat over $S$


regex_match $f : X \to Y$
nn candidate X \to
nn candidate a morphism
nn a morphism
new_annotation [(256083, 256096, 'VAR'), (256100, 256110, 'TYPE')]
===== sent In particular this implies that $Y$ is integral, see
Properties, Lemma \ref{properties-lemma-characterize-integral}.
Let $X_\eta = X_{1, \eta} \cup \ldots \cup X_{n, \eta}$
be the decomposition of $X_\eta$ into irreducible components.

new_annotation []
===== sent Let $X_i \subset X$ be the reduced closed subscheme whose generic
fibre is $X_{i, \eta}$. Note that $Z_{i, j} = X_i \cap X_j$
is a closed subset of $X_i$ whose generic fibre $Z_{i, j, \eta}$
is nowhere dense in $X_{i, \eta}$. ========

regex_match $X_i \subset X$
nn candidate the reduced closed subscheme
nn the reduced closed subscheme
new_annotation [(256725, 256740, 'VAR'), (256744, 256772, 'TYPE')]
===== sent \end{proof}

\begin{lemma}
\label{lemma-base-change-fibres-geometrically-irreducible}
Let $f : X \to Y$ be a morphism of schemes.
Let $g : Y' \to Y$ be any m

regex_match $X \to S$
nn candidate $X
nn candidate a smooth morphism
nn a smooth morphism
new_annotation [(371915, 371924, 'VAR'), (371928, 371945, 'TYPE')]
===== sent Let $x \in X$ with image $s \in ========

new_annotation []
===== sent \begin{lemma}
\label{lemma-slice-smooth-given-element}
Let $f : X \to S$ be a morphism of schemes.

regex_match $f : X \to S$
nn candidate X \to
nn candidate a morphism
nn a morphism
new_annotation [(372959, 372972, 'VAR'), (372976, 372986, 'TYPE')]
===== sent Let $x \in X$ be a point with image $s \in ========

regex_match $x \in X$
nn candidate a point
nn a point
new_annotation [(373003, 373012, 'VAR'), (373016, 373023, 'TYPE')]
===== sent S$.
Let $h \in \mathfrak m_x \subset \mathcal{O}_{X, x}$.
Assume
\begin{enumerate}
\item $f$ is smooth at $x$, and
\item the image $\text{d}\overline{h}$ of $\text{d}h$ in
$$
\Omega_{X_s/s, x} \otimes_{\mathcal{O}_{X_s, x}} \kappa(x) =
\Omega_{X/S, x} \otimes_{\mathcal{O}_{X, x}} \kappa(x)
$$
is nonzero.

new_anno

new_annotation [(506106, 506115, 'VAR'), (506119, 506126, 'TYPE')]
===== sent \end{proof}

\begin{theorem}[Stein factorization; general case]
\label{theorem-stein-factorization-general}
Let $S$ be a scheme.

regex_match $S$
nn candidate a scheme
nn a scheme
new_annotation [(507988, 507991, 'VAR'), (507995, 508003, 'TYPE')]
===== sent Let $f : X \to S$ be a proper morphism.

regex_match $f : X \to S$
nn candidate X \to
nn candidate a proper morphism
nn a proper morphism
new_annotation [(508009, 508022, 'VAR'), (508026, 508043, 'TYPE')]
===== sent Then $S' = \Spec(R')$.
Thus we may replace $S$ by $S'$ and assume that
$S = \Spec(R)$ is affine $R = \Gamma(X, \mathcal{O}_X)$.
Next, let $s \in S$ be a point. ========

regex_match $s \in S$
nn candidate a point
nn a point
new_annotation [(509334, 509343, 'VAR'), (509347, 509354, 'TYPE')]
===== sent Let $U \to S$ be an \'etale morphism
of affine schemes and let $u \in U$ be a point mapping to $s$.
Let $X_U \to U$ be the base change of $X$. By


Let $S$ be a scheme. ========

regex_match $S$
nn candidate a scheme
nn a scheme
new_annotation [(623660, 623663, 'VAR'), (623667, 623675, 'TYPE')]
===== sent Let $f : X \to Y$ be a morphism of schemes over $S$.
If $X$, $Y$ are weakly \'etale over $S$, then $f$ is weakly \'etale.

regex_match $f : X \to Y$
nn candidate $
nn candidate X \to
nn candidate a morphism
nn a morphism
new_annotation [(623681, 623694, 'VAR'), (623698, 623708, 'TYPE')]
===== sent Although the proof of the result is kind of
laborious, in essence it follows in a straightforward manner from
Epp's result on eliminating ramification, see
More on Algebra, Theorem \ref{more-algebra-theorem-epp}.

\medskip\noindent
Let $A$ be a Dedekind domain with fraction field $K$.
Let $X$ be a scheme flat and of finite type over $A$.

regex_match $A$
nn candidate a Dedekind domain
nn a Dedekind domain
new_annotation [(624878, 624881, 'VAR'), (624885, 624902, 'TYPE')]
===== sent Although the proof of the result is kind of
laborious, 

In [245]:
annotated_data_with_filenames = list(zip(annotated_data, list_of_texs))
random.shuffle(annotated_data_with_filenames)
train_data = [ann_data[0] for ann_data in annotated_data_with_filenames[:-1]]
test_data = [ann_data[0] for ann_data in annotated_data_with_filenames[-1:]] #we hold out one tex file for testing

In [246]:
def train_ner(nlp, train_data, entity_types):
    # Add new words to vocab.
    for raw_text, _ in train_data:
        doc = nlp.make_doc(raw_text)
        for word in doc:
            _ = nlp.vocab[word.orth]

    # Train NER.
    ner = EntityRecognizer(nlp.vocab, entity_types=entity_types)
    for itn in range(5):
        random.shuffle(train_data)
        for raw_text, entity_offsets in train_data:
            doc = nlp.make_doc(raw_text)
            gold = GoldParse(doc, entities=entity_offsets)
            ner.update(doc, gold)
    return ner

In [247]:
ner = train_ner(nlp, train_data, ['VAR', 'TYPE'])

In [249]:
#first test on a simple sentence
doc = nlp.make_doc('Let $S$ be a scheme and let $something here$ be a great thing you know.')  
nlp.tagger(doc)
ner(doc)

for ent in doc.ents:
    ent.merge()
    
for word in doc:
    if word.ent_type:
        print('(' + word.text + ':' + word.ent_type_ +')', end=word.whitespace_)
    else:
        print(word.text_with_ws, end='')
    


Let ($S$:VAR) be (a scheme:TYPE) and let ($something here$:VAR) be (a great thing:TYPE) you know.

In [250]:
#then test on the hold out tex file and save in a new tex file
doc = nlp.make_doc(annotated_data_with_filenames[-1][0][0])  
nlp.tagger(doc)
ner(doc)

for ent in doc.ents:
    ent.merge()

new_filename = 'a-' + annotated_data_with_filenames[-1][1] 
with open( 'annotated_tex_files/'+ new_filename, 'w') as f:
    for word in doc:
        if word.ent_type:
            f.write('(' + word.text + ':' + word.ent_type_ +')' + word.whitespace_)
        else:
            f.write(word.text_with_ws)